# 1. KoGPT 모델 분해하기


* 학습 목표
    * Ko GPT에 사용된 GPT-J 모델을 모듈 단위로 분해하여 언어모델의 구조 파악
* 환경 필요사항
    * KaKao-Brain에서 공개한 Ko-GPT3 모델을 활용
    * 일종의 경량화 버전인 float-16 버전의 모델을 활용하여도 Colab 무료 사용으로는 실습 제한
    * 원활한 실습을 위해서는 월 9.99 달러의 Colab Pro를 활용하는것을 권장

In [ ]:
# 필요 라이브러리 설치
!pip install -q transformers accelerate

## 필요 라이브러리 및 토크나이저와 모델 불러오기
- HuggingFace를 활용하여 토크나이저 및 사전학습모델을 다운로드 합니다.
- [카카오 KoGPT](https://github.com/kakaobrain/kogpt)에서 제공하는 [float16 버전](https://huggingface.co/kakaobrain/kogpt/tree/KoGPT6B-ryan1.5b-float16)을 사용했습니다. (13GB 정도의 디스크 공간이 필요합니다.)

In [ ]:
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm.auto import tqdm

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
  'kakaobrain/kogpt', revision = 'KoGPT6B-ryan1.5b-float16',
  torch_dtype = torch.float16,
)

# Evaluate Mode Setting
model.to('cuda')
model.eval()
print('Inference is Ready')

## 토크나이저 돌려보기

- 토크나이저는 [PreTrainedTokenizerFast](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.PreTrainedTokenizerFast)로 추상화되어있습니다.

In [ ]:
print(type(tokenizer))
print(tokenizer.special_tokens_map)
print("vocab_size:", tokenizer.vocab_size)
print("[BOS]:{} [EOS]:{} [PAD]:{} [UNK]:{}".format(tokenizer.bos_token_id, tokenizer.eos_token_id, tokenizer.pad_token_id, tokenizer.unk_token_id))

- [`tokenizer()`](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.PreTrainedTokenizer.__call__)를 호출하면 토큰화한 결과를  `input_ids`로 돌려줍니다.
- `input_ids`는 각 토큰의 id 입니다.

In [ ]:
text = "우리는 알아야 한다.[EOS]"
encoding = tokenizer(text)
print(encoding)

In [ ]:
# decode를 사용하면 문자열로 돌아올 수 있습니다.
print(tokenizer.decode(encoding['input_ids']))

# 각 토큰의 내용도 확인 할 수 있습니다.
print([tokenizer.decode([token]) for token in encoding['input_ids']])

In [ ]:
# batch 단위로 encoding, decoding도 가능합니다.
texts = ["우리는 알아야 한다.[EOS]", "우리는 알게 될 것이다.[EOS]"]
encoding = tokenizer(texts)
print(encoding['input_ids'][0])
print(encoding['input_ids'][1])
print(tokenizer.batch_decode(encoding['input_ids']))

In [ ]:
# 긴 샘플에 맞게 길이가 일치하도록 padding 할 수 있습니다.
# 결과를 tensor형태로 반환 할 수 있습니다.
tokenizer.padding_side = 'left'
encoding = tokenizer(texts, padding='longest', return_tensors='pt')
print(encoding['input_ids'])
print(tokenizer.batch_decode(encoding['input_ids']))

- 이외에도 너무 긴 문장을 잘라내는 max_length, truncation등 다양한 옵션이 있습니다.
- 추가적인 내용은 [PreTrainedTokenizerFast](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.PreTrainedTokenizerFast)를 참고하세요.

## 모델 돌려보기

In [ ]:
text = "우리는 알아야 한다.[EOS]"
encoding = tokenizer(text, return_tensors='pt')
outputs = model(
    input_ids = encoding['input_ids'].to('cuda'),
    labels = encoding['input_ids'].to('cuda'),
)

- 모델은 출력은 [CausalLMOutputWithPast](https://huggingface.co/docs/transformers/v4.27.1/en/main_classes/output#transformers.modeling_outputs.CausalLMOutputWithPast) 형태입니다.
- logits: 각 위치에서 그보다 앞쪽의 내용만 고려할 때 각 토큰이 등장할 확률을 나타내는 logit입니다.
- past_key_values: 사용된 key, value값입니다. 연속해서 다음 토큰을 생성하는 경우 반복되는 연산을 줄이기 위해 사용됩니다.
- loss: 정답 labels를 입력에 같이 넣어준 경우 logits과 비교한 [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)를 계산해서 돌려줍니다.

In [ ]:
print(list(outputs))
print(outputs['logits'].shape)

In [ ]:
n_layer = len(outputs['past_key_values'])
print("n_layer=", n_layer)
key, value = outputs['past_key_values'][0]
# 각 레이어의 key, value는 (batch_size, num_head, sequence_lenght, head_size) 형태입니다.
print(key.shape, value.shape)

## 모델 구조

In [ ]:
model.config

In [ ]:
pd.DataFrame([
    (param.dtype, param.shape, name)
    for name, param in model.named_parameters()
])

GPTJ

## Ko GPT 모델 분해하기

### _module 메서드와 Transformer 모듈
* _module Method를 활용한 모델 분리 
* 모델을 가장 큰 모듈 단위로 분해해 보면 다음과 같이 ‘transformer', 'lm_head' 로 나뉘어짐


### Transformer 모듈 살펴보기
* GPT 언어모델이 핵심 작동방식중 하나인 Attention 을 구현한 부분이 Transformer모듈
* Transformer 부분만 살펴보기 위해 다음과 같이 Key값을 입력하면 Transformer 부분에 해당하는 모델들이 출력
* 해당 모듈의 Key를 비롯한 속성 값들을 살펴보면 총 28개에 달하는 GPT Block으로 이루어져 있음을 알 수 있음

![nn](Ko_GPT_Moduel_Structure.png)

In [ ]:
print(model._modules.keys())

In [ ]:
model._modules['transformer']

In [ ]:
model._modules['transformer']._modules.keys()

In [ ]:
model._modules['transformer']._modules['wte']

In [ ]:
model._modules['transformer']._modules['drop']

In [ ]:
model._modules['transformer']._modules['ln_f']

In [ ]:
model._modules['transformer']._modules['h']._modules.keys()

In [ ]:
model._modules['transformer']._modules['h']._modules['1']

## Q K V 행렬 출력하기

### 각 GPT 블럭의 attn 모듈을 살펴보기
* attn 모듈을 찾아보면 결국 Query Key Value 행렬로 구성되어 있음
* 이를 출력하여 보면 4096 4096 의 크기를 갇는 정방 행렬임

In [ ]:
model._modules['transformer']._modules['h']._modules['1']._modules['attn']

In [ ]:
model._modules['transformer']._modules['h']._modules['1']._modules['attn']._modules['k_proj'].weight

In [ ]:
model._modules['transformer']._modules['h']._modules['1']._modules['attn']._modules['k_proj'].weight.shape